In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
#device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
#Hyper=parameter
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [ ]:
#data preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                               std = (0.5, 0.5, 0.5))])

In [ ]:
#Cifar10: 60000 32x32 color images
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=False,
                                             download=True,
                                             transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 48730850.05it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
#batch단위로 데이터를 제공

In [ ]:
#모델 정의
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 5) #3:RGB, 6:필터 수(지정), 5:5x5의 필터 사용
    self.pool = nn.MaxPool2d(2, 2) #(사이즈(2x2), 스트라이드 (2칸씩))
    self.conv2 = nn.Conv2d(6, 16, 5) #6:위애서 필터 수가 6이므로 인채널도6, 16:필터 수
    self.fc1 = nn.Linear(16*5*5, 100)
    self.fc2 = nn.Linear(100, 50)
    self.fc3 = nn.Linear(50, 10) #Cifar10의 클래스가 10이므로 10

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5) #Flatten #-1:batch
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
#((W - F + 2*P) / S) + 1 for Conv
#(( W - F ) / s) + 1 for Pooling

In [ ]:
train_dataset[0][0].shape

torch.Size([3, 32, 32])

In [ ]:
((32 - 5) / 1) + 1 #(6, 28, 28) Conv1

28.0

In [ ]:
((28 - 2) / 2) +1 #(6, 14, 14) Pool

14.0

In [ ]:
((14 - 5) / 1) + 1 #(16, 10, 10) Conv2

10.0

In [ ]:
((10 - 2) / 2) +1 #(16, 5, 5) Pool

In [ ]:
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [ ]:
#train

n_total_steps = len(train_loader)
for epoch in range(num_epochs): #에폭을 다음 만큼 반복
  for i, (images, labels) in enumerate(train_loader): #배치사이즈로 공급
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(i+1) % 3000 == 0:
      print(f'Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{n_total_steps}, Loss: {loss.item(): 3f}')

print("Finished Training!")



Epoch: 1/5, Step: 3000/12500, Loss:  2.317857
Epoch: 1/5, Step: 6000/12500, Loss:  2.319219
Epoch: 1/5, Step: 9000/12500, Loss:  2.296466
Epoch: 1/5, Step: 12000/12500, Loss:  2.297772
Epoch: 2/5, Step: 3000/12500, Loss:  2.263310
Epoch: 2/5, Step: 6000/12500, Loss:  2.100027
